In [1]:
from IPython.display import display, Math, HTML
from sympy.physics.quantum import TensorProduct
from symboliq.dirac_notation import DiracNotation, cx, h, i, ket_0, x, ket_1

In [5]:
dirac_notation = DiracNotation(x * ket_0)

In [6]:
res = dirac_notation.get_steps()

In [7]:
print(res)

(0) (|0><1| + |1><0|)*|0>
(1) |0><1|*|0> + |1><0|*|0>
(2) |1>



In [8]:
res = dirac_notation.get_steps_latex()

In [9]:
Math(res)

<IPython.core.display.Math object>